In [1]:
import glob
import os
import pandas as pd
import re

In [2]:
yearFolders = ['2007', '2008', '2009','2010', '2011', '2012', '2013', '2014', '2015','2016']
all_colnames = []

#for each year folder, concatenate across the 4 data profile files, 
    #save one resulting df per year in the dem_files_byYear list

for folder in yearFolders:
    dem_files = glob.glob(os.path.join('data/raw/ACS_2007_2016',folder,'ACS_*_1YR_*with_ann.csv*'))  # * allows for gziped csv's 
    year_pat = re.compile(r'(\d{2})')
    cur_year_colnames = []
    for fname in dem_files:
        dem_yr_df = pd.read_csv(fname, skiprows=1, header=0)
        dem_yr_df = dem_yr_df[dem_yr_df.columns.drop(list(dem_yr_df.filter(regex='Margin of Error')))]
        bad_col_names = [x for x in dem_yr_df.columns.values if 'Percent;' not in x]
        dem_yr_df = dem_yr_df[dem_yr_df.columns.drop(bad_col_names)]
        cur_year_colnames.append(dem_yr_df.columns.values)
    all_colnames.append(cur_year_colnames)

In [3]:
import csv
with open('data/derived/colnames.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile)
    for cur_year_colnames in all_colnames:
        flat_colnames = [x for sublist in cur_year_colnames for x in sublist]
        spamwriter.writerow(flat_colnames)


In [4]:
col_name2final_name={}
with open('data/derived/colnames_final.csv', 'r') as csvfile:
    spamwriter = csv.reader(csvfile)
    for row in spamwriter:
        final_name=row[-1]
        for colname in row:
            col_name2final_name[colname]=final_name

In [5]:
col_name2final_name
#this is the dictionary we want : )

{'': 'Percent; CITIZEN, VOTING AGE POPULATION - Citizen, 18 and over population - Female',
 'Percent; EMPLOYMENT STATUS - In labor force - Civilian labor force.1': 'Percent; EMPLOYMENT STATUS - Females 16 years and over - In labor force - Civilian labor force',
 'Percent; RACE - One race': 'Percent; RACE - One race',
 'Percent; INCOME AND BENEFITS (IN 2010 INFLATION-ADJUSTED DOLLARS) - $35,000 to $49,999': 'Percent; INCOME AND BENEFITS (IN 2016 INFLATION-ADJUSTED DOLLARS) - Total households - $35,000 to $49,999',
 'Percent; Estimate; SEX AND AGE - Total population - 65 years and over': 'Percent; SEX AND AGE - 65 years and over',
 'Percent; Estimate; SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI) - Housing units with a mortgage (excluding units where SMOCAPI cannot be computed) - 35.0 percent or more': 'Percent; SELECTED MONTHLY OWNER COSTS AS A PERCENTAGE OF HOUSEHOLD INCOME (SMOCAPI) - Housing units with a mortgage (excluding units where SMOCAPI cannot be c